# Udacity Data Investigation Project

# Movies Dataset Investigation

### Purpose

Conduct data analysis on movie dataset and create a file to share that documents your findings on Udacity classroom, unit 2 Project: Investigate a Dataset.

### Introduction

This movie dataset was supplied by Udacity, called IMDb movie data:

[IMDb movie data](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/July/5b57919a_data-set-options/data-set-options.pdf)

RangeIndex: 10866 entries, 0 to 10865

Data columns (total 21 columns):

dtypes: float64(4), int64(6), object(11)

### Process
This project uses Pandas, NumPy, Matplotlib, and other libraries to wrangle, explore, analyze and communicate data.

- 3 spearate .py files for functions used in the ipynb files

### Acknowledgments

- Stackoverflow
- Udacity lessons
- Data Validation Project
- GitHub/unionai-oss/pandera

### Hypothesis Questions
- What 5 year period were the most films released?
- What years were each genre at it's highest popularity?
- Are some genres more popular per 5 years? 
- What genre generates the most money?
- What is the most common day of the week that a movie is released? 
- Do movies generate more revenue if they are released certain days or on a holiday?

## Movies dataset exploration

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Read csv files into dataframe, view the first few rows
mdf=pd.read_csv('data/movies.csv')
mdf.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


In [3]:
mdf.tail()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/66,11,7.4,1966,0.000000,0.0
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/66,20,5.7,1966,0.000000,0.0
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94,Mystery|Comedy,Mosfilm,1/1/66,11,6.5,1966,0.000000,0.0
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80,Action|Comedy,Benedict Pictures Corp.,11/2/66,22,5.4,1966,0.000000,0.0
10865,22293,tt0060666,0.035919,19000,0,Manos: The Hands of Fate,Harold P. Warren|Tom Neyman|John Reynolds|Dian...,NaN,Harold P. Warren,It's Shocking! It's Beyond Your Imagination!,...,A family gets lost on the road and stumbles up...,74,Horror,Norm-Iris,11/15/66,15,1.5,1966,127642.279154,0.0


In [4]:
# View data types and shape, notice missing values
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [5]:
mdf.shape

(10866, 21)

In [6]:
#Explore Data types
mdf.dtypes

id                        int64
imdb_id                  object
popularity              float64
budget                    int64
revenue                   int64
original_title           object
cast                     object
homepage                 object
director                 object
tagline                  object
keywords                 object
overview                 object
runtime                   int64
genres                   object
production_companies     object
release_date             object
vote_count                int64
vote_average            float64
release_year              int64
budget_adj              float64
revenue_adj             float64
dtype: object

In [7]:
# View release date dtype
mdf['release_date'].head()

0      6/9/15
1     5/13/15
2     3/18/15
3    12/15/15
4      4/1/15
Name: release_date, dtype: object

In [8]:
# count unique years present in dataset
year_count= len(list(mdf['release_year'].unique()))
year_count

56

In [9]:
# View totals with missing cast
no_cast=mdf[mdf['cast'].isna()]
no_cast.count()

id                      76
imdb_id                 76
popularity              76
budget                  76
revenue                 76
original_title          76
cast                     0
homepage                28
director                70
tagline                 26
keywords                45
overview                76
runtime                 76
genres                  75
production_companies    36
release_date            76
vote_count              76
vote_average            76
release_year            76
budget_adj              76
revenue_adj             76
dtype: int64

In [10]:
# Explore null values in dataset
null_col=mdf.isnull().sum()

In [11]:
# list of all columns with null values
all_null_col= [k for k, v in null_col.items() if v >0]
all_null_col

['imdb_id',
 'cast',
 'homepage',
 'director',
 'tagline',
 'keywords',
 'overview',
 'genres',
 'production_companies']

In [12]:
# Find duplicate rows
mdf[mdf.duplicated()]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
2090,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0


In [13]:
# find all columns with multiple values | separated
for c in list(mdf.columns):
    v=mdf[c].iloc[0]
    if isinstance(v, str) and '|' in v:
        newline = '/n'
        print(f'{c} column has multiple values:')
        print('--------------------------------')

cast column has multiple values:
--------------------------------
keywords column has multiple values:
--------------------------------
genres column has multiple values:
--------------------------------
production_companies column has multiple values:
--------------------------------


In [14]:
mdf['keywords'].unique()

array(['monster|dna|tyrannosaurus rex|velociraptor|island',
       'future|chase|post-apocalyptic|dystopia|australia',
       'based on novel|revolution|dystopia|sequel|dystopic future', ...,
       'car race|racing|formula 1', 'car|trolley|stealing car',
       'fire|gun|drive|sacrifice|flashlight'], dtype=object)

In [15]:
mdf['genres'].unique()

array(['Action|Adventure|Science Fiction|Thriller',
       'Adventure|Science Fiction|Thriller',
       'Action|Adventure|Science Fiction|Fantasy', ...,
       'Adventure|Drama|Action|Family|Foreign',
       'Comedy|Family|Mystery|Romance',
       'Mystery|Science Fiction|Thriller|Drama'], dtype=object)

### Explore categories, popularity, revenue, and year

- separate categories for production companies and genres
- determine unique category names

In [16]:
# get list of unique genre names
%run categories.py
mdf['genres']= replace_pipe(mdf, 'genres')
genre=get_unique_list(mdf, 'genres')
len(genre)

21

In [17]:
# get list of unique production company names
mdf['production_companies']= replace_pipe(mdf, 'production_companies')
prod=get_unique_list(mdf, 'production_companies')
len(prod)

7890

In [18]:
# Revenue, budget
mdf['revenue'].describe()

count    1.086600e+04
mean     3.982332e+07
std      1.170035e+08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.400000e+07
max      2.781506e+09
Name: revenue, dtype: float64

In [19]:
# Revenue, budget
mdf['revenue_adj'].describe()

count    1.086600e+04
mean     5.136436e+07
std      1.446325e+08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      3.369710e+07
max      2.827124e+09
Name: revenue_adj, dtype: float64

In [20]:
# Revenue, budget
mdf['popularity'].describe()

count    10866.000000
mean         0.646441
std          1.000185
min          0.000065
25%          0.207583
50%          0.383856
75%          0.713817
max         32.985763
Name: popularity, dtype: float64

> END EXPLORE DATASET

## Dataset wrangling and validation

In [21]:
from pandera import Column, DataFrameSchema
pd.options.mode.chained_assignment = None  # default='warn'
from pathlib import Path
import yaml
import black
import sys
import logging
import numpy as np
import pandera as pd
%run validate.py
%run categories.py

In [22]:
# Create schema to keep track of columns of how I want data shaped
schema = pa.DataFrameSchema(
    {
    'popularity': Column(pa.Float64),
    'budget': Column(pa.Int64),
    'revenue': Column(pa.Int64),
    'net' : Column(pa.Int64),
    'original_title': Column(pa.String), 
    'director': Column(pa.String),
    'runtime': Column(pa.Int64),
    'action': Column(pa.Int),
    'adventure': Column(pa.Int),
    'animation': Column(pa.Int),
    'comedy': Column(pa.Int),
    'crime': Column(pa.Int),
    'documentary': Column(pa.Int),
    'drama': Column(pa.Int),
    'family': Column(pa.Int),
    'fantasy': Column(pa.Int),
    'foreign': Column(pa.Int),
    'history': Column(pa.Int),
    'horror': Column(pa.Int),
    'music': Column(pa.Int),
    'mystery': Column(pa.Int),
    'romance': Column(pa.Int),
    'science_fiction': Column(pa.Int),
    'tv_movie': Column(pa.Int),
    'thriller': Column(pa.Int),
    'war': Column(pa.Int),
    'western': Column(pa.Int),
    'no_genre' : Column(pa.Int),
    'production_companies': Column(pa.String),
    'release_date': Column(pa.DateTime),
    'release_day' : Column(pa.String),
    'holiday' : Column(pa.Bool),
    'vote_count': Column(pa.Int64),
    'vote_average': Column(pa.Float64),
    'release_year': Column(pa.Int64),
    'release_year_groups' : Column(pa.Category),
    'budget_adj' : Column(pa.Float64),
    'revenue_adj' : Column(pa.Float64),
    'net_adj' : Column(pa.Float64)
}

)

In [23]:
# Put Schema into YAML file
yaml_sche = schema.to_yaml()
f = Path('file.yml')
f.touch()
f.write_text(yaml_sche)

5535

In [24]:
# open the yaml file
with open('file.yml', 'r') as f:
    file = yaml.safe_load(f)

In [25]:
# check for duplicates
mdf[mdf.duplicated()]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
2090,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,"Crime,Drama,Action,Thriller,Science Fiction","Namco,Light Song Films",3/20/10,110,5.0,2010,30000000.0,967000.0


In [26]:
# drop dulicates
mdf.drop_duplicates()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,"Action,Adventure,Science Fiction,Thriller","Universal Studios,Amblin Entertainment,Legenda...",6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,"Action,Adventure,Science Fiction,Thriller","Village Roadshow Pictures,Kennedy Miller Produ...",5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,"Adventure,Science Fiction,Thriller","Summit Entertainment,Mandeville Films,Red Wago...",3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,"Action,Adventure,Science Fiction,Fantasy","Lucasfilm,Truenorth Productions,Bad Robot",12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,"Action,Crime,Thriller","Universal Pictures,Original Film,Media Rights ...",4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/66,11,7.4,1966,0.000000e+00,0.000000e+00
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176,"Action,Adventure,Drama","Cherokee Productions,Joel Productions,Douglas ...",12/21/66,20,5.7,1966,0.000000e+00,0.000000e+00
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94,"Mystery,Comedy",Mosfilm,1/1/66,11,6.5,1966,0.000000e+00,0.000000e+00
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80,"Action,Comedy",Benedict Pictures Corp.,11/2/66,22,5.4,1966,0.000000e+00,0.000000e+00


In [27]:
# change NaN to Independent in production companies column
mdf['production_companies']=mdf['production_companies'].replace(np.nan,'Independent')
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  10866 non-null  object 
 15  release_date       

In [28]:
# run column validation
col_val(mdf)

column name and column length validation failed
Following File columns are not in the YAML file ['keywords', 'cast', 'imdb_id', 'id', 'tagline', 'homepage', 'revenue', 'budget', 'overview', 'genres']
Following YAML columns are not in the file uploaded ['documentary', 'music', 'fantasy', 'war', 'net_adj', 'tv_movie', 'crime', 'horror', 'romance', 'comedy', 'history', 'no_genre', 'release_day', 'holiday', 'science_fiction', 'mystery', 'thriller', 'family', 'animation', 'action', 'foreign', 'adventure', 'drama', 'western', 'release_year_groups']


0

In [29]:
# Remove columns from movies dataframe that will not be used in analysis and is not in the schema
# leave genres in for now
mdf= mdf.drop(['budget', 'revenue', 'overview', 'imdb_id', 'tagline', 'id', 'homepage', 'cast', 'keywords'], axis=1)     

In [30]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   popularity            10866 non-null  float64
 1   original_title        10866 non-null  object 
 2   director              10822 non-null  object 
 3   runtime               10866 non-null  int64  
 4   genres                10843 non-null  object 
 5   production_companies  10866 non-null  object 
 6   release_date          10866 non-null  object 
 7   vote_count            10866 non-null  int64  
 8   vote_average          10866 non-null  float64
 9   release_year          10866 non-null  int64  
 10  budget_adj            10866 non-null  float64
 11  revenue_adj           10866 non-null  float64
dtypes: float64(4), int64(3), object(5)
memory usage: 1018.8+ KB


In [31]:
# check columns
col_val(mdf)

column name and column length validation failed
Following File columns are not in the YAML file ['genres']
Following YAML columns are not in the file uploaded ['documentary', 'music', 'fantasy', 'war', 'net_adj', 'tv_movie', 'crime', 'horror', 'romance', 'comedy', 'history', 'no_genre', 'release_day', 'holiday', 'science_fiction', 'mystery', 'thriller', 'family', 'animation', 'action', 'foreign', 'adventure', 'drama', 'western', 'release_year_groups']


0

Leave genres for now, begin adding columns for analysis.

In [32]:
# Add columns to movies dataframe that are in the schema
mdf['net_adj'] = mdf['revenue_adj']-mdf['budget_adj']

Separating Categories from genres columns

- separate categories in genres
- create new columns for genres using 0, 1

In [33]:
# replace | with ,
mdf['genres']=replace_pipe(mdf, 'genres')

In [34]:
# List of unique genres
u_g=get_unique_list(mdf, 'genres')
len(u_g)

21

In [35]:
# add new genre columns to mdf, set all values to zero
mdf=insert_zero(mdf, u_g)

In [36]:
# Change new column value to 1 if category is present in old column. 
for g in u_g:
    category_present(mdf, 'genres', g)

NameError: name 'category_present' is not defined

In [ ]:
# Schema check
%run validate.py
col_val(mdf)

In [ ]:
# drop old genres
mdf.drop(['genres'], axis=1, inplace=True)

In [ ]:
# rename nan
mdf.rename(columns = {'nan':'no_genre'}, inplace = True)

In [ ]:
# Infor check
mdf.info()

In [ ]:
# column name check
col_val(mdf)

### Clean-Shape Dates 
- Change release_date to datetime
- Add release days and holidays columns
- Get 5 year categories for release years

In [ ]:
# Fixing dates in release date column to yyyy-mm-dd
# split of the values in release_day column to 3 seperate columns
date_fix=mdf['release_date'].str.split('/', expand=True)

# 0-month, 1-day, 2-year
# fill zeros for month and day
date_fix[0]=date_fix[0].str.zfill(2)
date_fix[1]=date_fix[1].str.zfill(2)

In [ ]:
# Fix yyyy format column
date_fix[2]=date_fix[2].astype(int)
# iterate over value and create yyyy
for k, v in date_fix[2].iteritems():
    if v in range(0, 22):
        date_fix[2].iloc[k]=v+2000
    else:
        date_fix[2].iloc[k]=v+1900
       
date_fix[2]=date_fix[2].astype(str)

In [ ]:
# bring the dates back together
date_fix['new']= date_fix[2]+date_fix[0]+date_fix[1]
# view one of the rows
date_fix['new'][9]

In [ ]:
# change new column to datetime format
date_new=pd.to_datetime(date_fix['new'])
date_new.info

In [ ]:
# replace release date column in new datetime format
mdf['release_date']=date_new
mdf['release_date']

In [ ]:
# dataframe check
mdf.info()

In [ ]:
# drop null values for director
# Drop rows if value in director column is null
mdf=mdf.dropna(subset=['director'])
mdf.info()

In [ ]:
col_val(mdf)

In [ ]:
# run datatype validation
dtype_val(mdf)

In [ ]:
# New release day column, indicates day of the week the movie was released
mdf['release_day']=pd.to_datetime(mdf['release_date']).dt.day_name()

In [ ]:
# Create holiday column to indicate if the release day was a holiday
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal=calendar()
holidays=cal.holidays(start=mdf['release_date'].min(), end=mdf['release_date'].max())
mdf['holiday'] = mdf['release_date'].isin(holidays)

In [ ]:
# just curious, checkout holiday count
mdf['holiday'].value_counts()

In [ ]:
# Add release year grouping, evey 5 years
mdf['release_year_groups'] = pd.cut(mdf['release_year'], 11)
mdf['release_year_groups']

In [ ]:
# columns check
col_val(mdf)

In [ ]:
# run datatype validation
dtype_val(mdf)

In [ ]:
# put reshaped dataframe into out.csv
mdf.to_csv('out.csv')

In [ ]:
# read new out csv file, view columns
out=pd.read_csv('out.csv', index_col=[0])
out.columns

## Visualizing  and Analyzing the Movies Dataset

- What 5 year period were the most films released?
- Are some genres more popular per 5 years? 
- What is the most common day of the week that a movie is released? 
- Do movies generate more revenue if they are released certain days or on a holiday?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%run group.py
df=pd.read_csv('out.csv')

Most movies are released in 2010-2015

In [ ]:
# View plot of release year count
df['release_year'].value_counts().plot(title='Number of Films Released between 1959-2015');

2010-2015 had the most released films, 3099, for this dataset that spans 56 years.

In [ ]:
df['release_year_groups'].value_counts()

In [ ]:
df['release_year_groups'].value_counts().plot.pie(title='Film % totals: 5 Year Groups',figsize=(7, 7),autopct='%1.1f%%', label='');

### Movie Genre Analysis
Note: All movie genres, per movie, were included with this analysis.

In [ ]:
# get lists of unique genres and unique 5 year groups
genres_list=['action','adventure','animation','comedy','crime','documentary','drama','family','fantasy','foreign','history','horror','music','mystery','romance','science_fiction','tv_movie','thriller','war','western']
release_year_groups_list=df['release_year_groups'].unique().tolist()
release_year_groups_list

### Genre-year category analysis

In [ ]:
# get function to make year-genre dataframes
def dict(yrgp):
    d=[]
    for g in genres_list:
        sum=df.loc[df['release_year_groups']==yrgp][g].sum()
        d.append({
            'year category': yrgp, 'genre': g, 'sums': sum
        })
    return pd.DataFrame(d)
    

In [ ]:
# make a list of dataframes
dlist = [ dict(yrgp) for yrgp in release_year_groups_list]


### Drama was the most common type of genre film made in all year categories except for 1985-1990, which was Comedy.

In [ ]:
# Genreate bar charts for most common genre per year category
i=0
for d in dlist:
    title=dlist[i]['year category'][1]
    max=d['genre'][d['sums'].idxmax()]
    dlist[i].plot(kind='bar', x='genre', y='sums', title=title);
    print(f'{title} most common genre: {max}')
    i=i+1

### Finding Genre best appearance per year

In [ ]:
# max number of movies released per genre 
row_list=[max_genre_by_year(g) for g in genres_list]
genre_max=pd.DataFrame(row_list, columns=['genres', 'max_released', 'release year'])

Year Categories 2010-2015, 2005-2010, 1965-1970 are where we find genre best appearance years.

In [ ]:
genre_max_s=genre_max.groupby(['release year']).size()
genre_max_s.plot.pie(title='Max Movies made per Genre/5 Years', label='');

- 14 Genres had most appearacnes between 2010-2015

In [ ]:
genre_max.loc[genre_max['release year']=='(2010.0, 2015.0]']

- 5 genres, Family, Fantasy, History and Romance Genres had their best showing between 2005-2010

In [ ]:
genre_max.loc[genre_max['release year']=='(2005.0, 2010.0]']

- Western Movie Genre had it's best showing between 1965-1970

In [ ]:
genre_max.loc[genre_max['release year']=='(1965.0, 1970.0]']

## Days of Week Released and Adjusted Revenue
- Friday is the most popular day for a movie to be released.
- Movies released on Wednesday have highest average adjusted revenue.

In [ ]:
# Get number of films released, by day of the week.
df.groupby(['release_day']).size().plot(kind='bar', title='Number of Movies Released per Day of the Week');

In [ ]:
df.groupby(['release_day'])['revenue_adj'].mean()

### Movies released on Wednesday have highest average adjusted revenue.

In [ ]:
# Plot Average Adjusted Revenueper Day of Week - Movie Release
df.groupby(['release_day'])['revenue_adj'].mean().plot(kind='bar', title='Average Adjusted Revenue per Day of the Week');

### Highest Adjusted Revenue Movie

In [ ]:
df.loc[df['revenue_adj'].idxmax(), ['original_title']]

## Holidays
- Most Holidays a movie was released land on Monday
- Highest Adjusted Revenue Holiday-days are Tuesdays

In [ ]:
# Display Days of Week totlals, if it's a Holiday
holiday=df[df['holiday']==True]
holiday.groupby(['release_day']).size().plot(kind='bar', title='Holiday count by Week day');

In [ ]:
# Determine Ave Revenue Generated on Holidays
holiday.groupby(['release_day'])['revenue_adj'].mean()

### Highest Adjusted Revenue Holiday-days are Tuesdays

In [ ]:
holiday.groupby(['release_day'])['revenue_adj'].mean().plot(kind='bar', title='Holiday Release Day Average Adjusted Revenue');

Final Thoughts:
- I was surprised that the most common movie genre is Drama
- Movies released on Wednesday have highest average adjusted revenue was surprising too.
- I enjoyed data collection, data cleaning, wrangling and validation the most.
- I alos enjoy working with categorical data, and is why I chose the movie dataset.